In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [71]:
import pandas as pd
import numpy as np

df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [72]:
df_pesoEaltura = df_pesoEaltura.rename(columns={"Record ID": "record_id",
                                                "Data: ": "data",
                                                "Peso": "peso",
                                                "Altura (em centimetros)": "altura"})

lista_RecorId = df_pesoEaltura['record_id'].unique()
df_novo = pd.DataFrame()

for i in lista_RecorId: 
    df_query = df_pesoEaltura.query(f'record_id == {i}')
    df_query = df_query.sort_values(by='data')

    i = 0

    if len(df_query) > 1: 
      df_row_second = df_query.iloc[1]

      numero = len(df_query) - 1

      for col in ['altura', 'peso', 'IMC']:
        while not(pd.isna(df_row_second[col].all())) and numero > 1: 
          df_row_second = df_query.iloc[len(df_query) - numero]
          numero -= 1
        while pd.isna(df_row_first[col].all()): 
          df_row_first = df_query.iloc[i]
          i += 1

      df_row_first['ultimo_peso'] = df_row_second['peso']
      df_row_first['ultima_altura'] = df_row_second['altura']
      df_row_first['ultimo_IMC'] = df_row_second['IMC']

      # df_row_first['data'] = df_row_first['data'].apply(pd.to_datetime)
      # df_row_second['data'] = df_row_second['data'].apply(pd.to_datetime)

      # df_row_first['diferenca_tempo'] =  (df_row_first['data'] - df_row_second['data']).dt.days
      
      df_novo = df_novo.append(df_row_first, ignore_index=True)
      
    else: 
      df_novo = df_novo.append(df_row_first, ignore_index=True)

df_novo = df_novo.rename(columns={
                        "peso": "primeiro_peso", 
                        "altura": "primeira_altura",
                        "IMC": "primeiro_IMC"
                        })

df_pesoEaltura = df_novo

In [73]:
np.random.seed(73)

df_pesoEaltura['primeiro_IMC'] = (df_pesoEaltura['primeiro_peso'] / pow(df_pesoEaltura['primeira_altura']/100, 2))

primeiro_peso_mean = df_pesoEaltura["primeiro_peso"].mean(skipna=True)
primeiro_peso_std = df_pesoEaltura["primeiro_peso"].std(skipna=True)
null_primeiro_peso = df_pesoEaltura.primeiro_peso.isnull().sum()
lista_normal_primeiro_peso = np.random.normal(primeiro_peso_mean, primeiro_peso_std, null_primeiro_peso)

primeira_altura_mean = df_pesoEaltura["primeira_altura"].mean(skipna=True)
primeira_altura_std = df_pesoEaltura["primeira_altura"].std(skipna=True)
null_primeira_altura = df_pesoEaltura.primeira_altura.isnull().sum()
lista_normal_primeira_altura = np.random.normal(primeira_altura_mean, primeira_altura_std, null_primeira_altura)

ultimo_peso_mean = df_pesoEaltura["ultimo_peso"].mean(skipna=True)
ultimo_peso_std = df_pesoEaltura["ultimo_peso"].std(skipna=True)
null_ultimo_peso = df_pesoEaltura.ultimo_peso.isnull().sum()
lista_normal_ultimo_peso = np.random.normal(ultimo_peso_mean, ultimo_peso_std, null_ultimo_peso)

ultima_altura_mean = df_pesoEaltura["ultima_altura"].mean(skipna=True)
ultima_altura_std = df_pesoEaltura["ultima_altura"].std(skipna=True)
null_ultima_altura = df_pesoEaltura.ultima_altura.isnull().sum()
lista_normal_ultima_altura = np.random.normal(ultima_altura_mean, ultima_altura_std, null_ultima_altura)

for i in range(0, null_primeiro_peso):
  lista_normal_primeiro_peso[i] = round(lista_normal_primeiro_peso[i])

for i in range(0, null_primeira_altura):
  lista_normal_primeira_altura[i] = round(lista_normal_primeira_altura[i])

for i in range(0, null_ultimo_peso):
  lista_normal_ultimo_peso[i] = round(lista_normal_ultimo_peso[i])

for i in range(0, null_ultima_altura):
  lista_normal_ultima_altura[i] = round(lista_normal_ultima_altura[i])

df_pesoEaltura["primeiro_peso"] = df_pesoEaltura["primeiro_peso"].fillna(pd.Series(lista_normal_primeiro_peso, index=df_pesoEaltura["primeiro_peso"].index[df_pesoEaltura["primeiro_peso"].isnull()]))
df_pesoEaltura["primeira_altura"] = df_pesoEaltura["primeira_altura"].fillna(pd.Series(lista_normal_primeira_altura, index=df_pesoEaltura["primeira_altura"].index[df_pesoEaltura["primeira_altura"].isnull()]))
df_pesoEaltura["ultimo_peso"] = df_pesoEaltura["ultimo_peso"].fillna(pd.Series(lista_normal_ultimo_peso, index=df_pesoEaltura["ultimo_peso"].index[df_pesoEaltura["ultimo_peso"].isnull()]))
df_pesoEaltura["ultima_altura"] = df_pesoEaltura["ultima_altura"].fillna(pd.Series(lista_normal_ultima_altura, index=df_pesoEaltura["ultima_altura"].index[df_pesoEaltura["ultima_altura"].isnull()]))


df_pesoEaltura['ultimo_IMC'] = (df_pesoEaltura['ultimo_peso'] / pow(df_pesoEaltura['ultima_altura']/100, 2))

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC
0,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,64.90,158.0,25.997436
1,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,63.80,155.0,26.555671
2,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,85.90,152.0,37.179709
3,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,51.00,143.0,24.940095
4,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,63.65,167.0,22.822618
...,...,...,...,...,...,...,...,...,...,...
4267,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,87.00,154.0,36.684095
4268,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,78.00,151.0,34.209026
4269,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,75.65,156.0,31.085634
4270,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,69.00,174.0,22.790329


In [75]:
df_pesoEaltura['diferenca_peso'] = df_pesoEaltura['ultimo_peso'] - df_pesoEaltura['primeiro_peso']
df_pesoEaltura['coef_peso'] = df_pesoEaltura['ultimo_peso'] / df_pesoEaltura['primeiro_peso']
df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['ultima_altura'] - df_pesoEaltura['primeira_altura']

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,diferenca_peso,coef_peso,diferenca_altura
0,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,64.90,158.0,25.997436,-15.45,0.807716,-3.0
1,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,63.80,155.0,26.555671,-16.55,0.794026,-6.0
2,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,85.90,152.0,37.179709,5.55,1.069073,-9.0
3,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,51.00,143.0,24.940095,-29.35,0.634723,-18.0
4,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,63.65,167.0,22.822618,-16.70,0.792159,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,87.00,154.0,36.684095,6.65,1.082763,-7.0
4268,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,78.00,151.0,34.209026,-2.35,0.970753,-10.0
4269,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,75.65,156.0,31.085634,-4.70,0.941506,-5.0
4270,82240.0,Dados Antropometricos,1.0,2020-12-08,80.35,161.0,30.998032,69.00,174.0,22.790329,-11.35,0.858743,13.0
